In [ ]:
!pip install numpy
!pip install pandas 
!pip install matplotlib

In [30]:
!pip install scipy

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 262.6 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 324.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/38.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.7 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.1/38.7 MB 1.8 MB/s eta 0:00:22
   ---------------------------------------- 0.2/38.7 MB 1.8 MB/s eta 0:00:22
   ---------------------------------------- 0.3/38.7 MB 1.7 MB/s eta 0:00:23
   ---------------------------------------- 0.4/38.7 MB 1.8 MB/s eta 0:00:21
   ---------------------------------------- 0.5/38.7 MB 1.8 MB/s eta 0:00:21
    --------------------------------------- 0.6/38.7 MB 2.0 MB/s eta 0:00:19
    --


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\vinic\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Frequências Características de falha
**Frequência devido à falha local na pista externa ($f_e$):**
$$
f_e = \frac{n f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha local na pista interna ($f_i$):**
$$
f_i = \frac{n f_r}{2} \left( 1 + \frac{d}{D} \cos \phi \right)
$$

**Frequência fundamental da carcaça ($f_C$):**
$$
f_C = \frac{f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha na esfera ($f_B$):**
$$
f_B = \frac{D f_r}{2d} \left[ 1 - \left( \frac{d}{D} \cos \phi \right)^2 \right]
$$

# Amplitudes de Falha em função do diâmetro da falha

#### Falha na Esfera

1730 RPM Esfera (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :------------- | :-------------- | :----------------- |
| 0.007"         | 0.000076        | 68.906512          |
| 0.014"         | 0.000052        | 67.007271          |
| 0.021"         | 0.000061        | 67.203773          |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000052 | 67.007271 |
| 1750 | 0.000048 | 66.988548 |
| 1772 | 0.000038 | 70.937316 |
| 1797 | 0.000027 | 71.894030 |


#### Falha na Pista Interna

Falha na Pista Interna a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.005900 | 154.933817 |
| 0.014" | 0.008288 | 155.495846 |
| 0.021" | 0.011044 | 155.421302 |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.008288 | 155.495846 |
| 1750 | 0.009055 | 157.674442 |
| 1772 | 0.009426 | 159.644141 |
| 1797 | 0.007026 | 161.712325 |

#### Pista Externa

Falha na Pista Externa a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.002440 | 103.384977 |
| 0.014" | 0.000360 | 103.286308 |
| 0.021" | 0.002292 | 102.991204 |


Falha na Pista Externa para diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000360 | 103.286308 |
| 1750 | 0.000369 | 104.886496 |
| 1772 | 0.000420 | 106.307723 |
| 1797 | 0.000364 | 107.644075 |


In [ ]:
import os
import pandas as pd
import numpy as np
import pprint

# =============================================================================
# BLOCO 1: CONFIGURAÇÃO, CARREGAMENTO E SEGMENTAÇÃO (APENAS DRIVE END)
# =============================================================================

# --- 1. CONFIGURAÇÕES GERAIS ---
caminho_raiz = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'
params_drive_end = {'n': 9, 'd': 0.3126, 'D': 1.537, 'phi_graus': 0.0}
TAXA_AMOSTRAL = 12000

# Dicionários de mapeamento
mapa_tipo_falha = {'IR': 'Pista Interna', 'B': 'Esfera', 'OR': 'Pista Externa', 'Normal': 'Normal'}
mapa_diametro_falha = {'7': '0.007"', '14': '0.014"', '21': '0.021"'}

# --- PARÂMETROS DE SEGMENTAÇÃO ---
tamanho_segmento = 4096
sobreposicao_percentual = 0.3
passo = int(tamanho_segmento * (1 - sobreposicao_percentual))

# --- 2. CARREGAMENTO E PROCESSAMENTO DOS ARQUIVOS ---
dicionario_de_dataframes = {}
print(f"Iniciando a leitura e segmentação dos arquivos (APENAS DRIVE END) em '{caminho_raiz}'...")

for pasta_atual, _, arquivos in os.walk(caminho_raiz):
    for nome_arquivo in arquivos:
        if nome_arquivo.endswith('.npz'):
            caminho_completo = os.path.join(pasta_atual, nome_arquivo)
            
            # Decodificação de metadados
            nome_sem_ext = nome_arquivo.replace('.npz', '')
            partes = nome_sem_ext.split('_')
            rpm = partes[0]
            
            if 'Normal' in nome_arquivo:
                tipo_falha_cod, diametro_falha = 'Normal', 'N/A'
            else:
                tipo_falha_cod = partes[1].split('@')[0]
                diametro_falha = mapa_diametro_falha.get(partes[2], 'Desconhecido')
            
            tipo_falha = mapa_tipo_falha.get(tipo_falha_cod, 'Desconhecido')
            
            # --- AJUSTE: Processar apenas o sensor 'DE' ---
            sensor_cod = 'DE'
            try:
                dados_npz = np.load(caminho_completo)
                if sensor_cod in dados_npz.files:
                    sinal_completo = dados_npz[sensor_cod].ravel()
                    
                    if 'Normal' in nome_arquivo:
                        chave_base = f"{nome_sem_ext}_{sensor_cod}"
                    else:
                        partes_chave = nome_sem_ext.split('_')
                        partes_chave[-1] = partes_chave[-1].rstrip('0123456789')
                        chave_base = "_".join(partes_chave)
                        
                    # Loop de segmentação
                    for i, inicio in enumerate(range(0, len(sinal_completo) - tamanho_segmento + 1, passo)):
                        segmento = sinal_completo[inicio : inicio + tamanho_segmento]
                        df_segmento = pd.DataFrame({'amplitude': segmento})
                        
                        # Adiciona metadados
                        df_segmento['arquivo_origem'] = nome_arquivo
                        df_segmento['rotacao_rpm'] = int(rpm)
                        df_segmento['tipo_falha'] = tipo_falha
                        df_segmento['diametro_falha'] = diametro_falha
                        df_segmento['local_sensor'] = 'Drive End'
                        
                        chave_segmento = f"{chave_base}_seg_{i}"
                        dicionario_de_dataframes[chave_segmento] = df_segmento

            except Exception as e:
                print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")

if not dicionario_de_dataframes:
    print(f"\nERRO: Nenhum segmento do Drive End foi carregado.")
else:
    print(f"\nProcesso concluído! {len(dicionario_de_dataframes)} SEGMENTOS do Drive End foram carregados.")
    chave_exemplo = list(dicionario_de_dataframes.keys())[0]
    print(f"\nExemplo de um segmento (chave: '{chave_exemplo}'):")
    print(dicionario_de_dataframes[chave_exemplo].head())
    print(len(dicionario_de_dataframes))

Iniciando a leitura e segmentação dos arquivos (APENAS DRIVE END) em 'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'...

Processo concluído! 3329 SEGMENTOS do Drive End foram carregados.

Exemplo de um segmento (chave: '1730_B_14_DE_seg_0'):
   amplitude      arquivo_origem  rotacao_rpm tipo_falha diametro_falha  \
0   0.204633  1730_B_14_DE12.npz         1730     Esfera         0.014"   
1   0.248395  1730_B_14_DE12.npz         1730     Esfera         0.014"   
2   0.042324  1730_B_14_DE12.npz         1730     Esfera         0.014"   
3  -0.015615  1730_B_14_DE12.npz         1730     Esfera         0.014"   
4  -0.058555  1730_B_14_DE12.npz         1730     Esfera         0.014"   

  local_sensor  
0    Drive End  
1    Drive End  
2    Drive End  
3    Drive End  
4    Drive End  
3329


In [32]:
# =============================================================================
# BLOCO DE GERAÇÃO DE DADOS SINTÉTICOS (INCLUINDO SINAIS NORMAIS)
# =============================================================================

# --- 1. FUNÇÃO PARA CÁLCULO DE FREQUÊNCIA TEÓRICA ---
def calcular_frequencias_rolamento(n, fr, d, D, phi_graus=0.0):
    """Calcula as frequências teóricas de falha de um rolamento."""
    phi_rad = np.deg2rad(phi_graus)
    termo_comum = (d / D) * np.cos(phi_rad)
    return {
        'Pista Externa': (n * fr / 2) * (1 - termo_comum),
        'Pista Interna': (n * fr / 2) * (1 + termo_comum),
        'Esfera': (D * fr / (2 * d)) * (1 - termo_comum**2)
    }

# --- 2. PARÂMETROS DE GERAÇÃO (APENAS DRIVE END) ---
amplitudes_referencia = {
    'Drive End': {'Esfera': 0.0001, 'Pista Interna': 0.001, 'Pista Externa': 0.0001}
}
multiplicadores = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 5.0, 10.0, 25.0, 50.0, 100.0]
fases_para_adicionar_rad = [
    0, np.pi/4, np.pi/2, 3*np.pi/4,
    np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4
]

# --- 3. IDENTIFICAÇÃO DOS SEGMENTOS NORMAIS ---
segmentos_normais = {
    chave: df for chave, df in dicionario_de_dataframes.items()
    if df['tipo_falha'].iloc[0] == 'Normal' and df['local_sensor'].iloc[0] == 'Drive End'
}
print(f"Encontrados {len(segmentos_normais)} segmentos normais do Drive End para usar como base.")

# --- 4. GERAÇÃO E COMBINAÇÃO DOS SINAIS ---
lista_dados_finais = []

# Loop para gerar os sinais com falha sintética
for chave_normal, df_normal in segmentos_normais.items():
    sinal_normal_base = df_normal['amplitude'].values
    rpm_atual = df_normal['rotacao_rpm'].iloc[0]
    
    N_PONTOS = len(sinal_normal_base)
    duracao_s = N_PONTOS / TAXA_AMOSTRAL
    t = np.linspace(0.0, duracao_s, N_PONTOS, endpoint=False)
    
    fr_hz = rpm_atual / 60
    freqs_teoricas = calcular_frequencias_rolamento(fr=fr_hz, **params_drive_end)
    
    for tipo_falha_sintetica in ['Pista Externa', 'Pista Interna', 'Esfera']:
        freq_teorica = freqs_teoricas[tipo_falha_sintetica]
        amp_ref = amplitudes_referencia['Drive End'][tipo_falha_sintetica]
            
        for mult in multiplicadores:
            for fase in fases_para_adicionar_rad:
                amplitude_final = amp_ref * mult
                sinal_falha_sintetico = amplitude_final * np.sin(2 * np.pi * freq_teorica * t + fase)
                sinal_final_combinado = sinal_normal_base + sinal_falha_sintetico
                
                lista_dados_finais.append({
                    'sinal_final': sinal_final_combinado,
                    'tipo_falha_adicionada': tipo_falha_sintetica,
                    'rpm': rpm_atual,
                    'multiplicador_amplitude': mult,
                    'fase_adicionada_rad': fase,
                    'base_normal': chave_normal,
                    'local_sensor': 'Drive End'
                })

# --- 5. ADIÇÃO DOS SEGMENTOS NORMAIS ORIGINAIS ---
print(f"\nAdicionando os {len(segmentos_normais)} segmentos normais ao conjunto de dados final...")
for chave_normal, df_normal in segmentos_normais.items():
    lista_dados_finais.append({
        'sinal_final': df_normal['amplitude'].values,
        'tipo_falha_adicionada': 'Normal', # Classe "Normal"
        'rpm': df_normal['rotacao_rpm'].iloc[0],
        'multiplicador_amplitude': 0, # Não aplicável
        'fase_adicionada_rad': 0, # Não aplicável
        'base_normal': chave_normal,
        'local_sensor': 'Drive End'
    })

# --- 6. DATAFRAME FINAL COM DADOS SINTÉTICOS E NORMAIS ---
df_final_completo = pd.DataFrame(lista_dados_finais)

print("\n\n--- Geração de Dados Concluída! ---")
print(f"Total de {len(df_final_completo)} segmentos (sintéticos + normais) gerados para o Drive End.")
print("\n--- Exemplo do DataFrame Final Gerado ---")
# Mostra as últimas linhas para vermos os exemplos de sinais normais
print(df_final_completo.drop(columns=['sinal_final']).tail())

Encontrados 588 segmentos normais do Drive End para usar como base.

Adicionando os 588 segmentos normais ao conjunto de dados final...


--- Geração de Dados Concluída! ---
Total de 169932 segmentos (sintéticos + normais) gerados para o Drive End.

--- Exemplo do DataFrame Final Gerado ---
       tipo_falha_adicionada   rpm  multiplicador_amplitude  \
169927                Normal  1797                      0.0   
169928                Normal  1797                      0.0   
169929                Normal  1797                      0.0   
169930                Normal  1797                      0.0   
169931                Normal  1797                      0.0   

        fase_adicionada_rad            base_normal local_sensor  
169927                  0.0  1797_Normal_FE_seg_79    Drive End  
169928                  0.0  1797_Normal_FE_seg_80    Drive End  
169929                  0.0  1797_Normal_FE_seg_81    Drive End  
169930                  0.0  1797_Normal_FE_seg_82    Drive End  

In [33]:
import numpy as np
import pandas as pd
from scipy.stats import skew

# =============================================================================
# BLOCO DE EXTRAÇÃO DE ATRIBUTOS DOS SINAIS SINTÉTICOS GERADOS
# =============================================================================

# --- 1. FUNÇÕES PARA CÁLCULO DOS ATRIBUTOS ---

# --- DOMÍNIO DO TEMPO ---
def calcular_tf2_std(sinal):
    """Calcula o Desvio Padrão (TF2)."""
    return np.std(sinal)

def calcular_tf3_rms(sinal):
    """Calcula o valor RMS (TF3)."""
    return np.sqrt(np.mean(sinal**2))

def calcular_tf4_fator_forma(sinal):
    """Calcula o Fator de Forma (TF4)."""
    rms = calcular_tf3_rms(sinal)
    media_abs = np.mean(np.abs(sinal))
    return rms / media_abs if media_abs != 0 else 0

# --- DOMÍNIO DA FREQUÊNCIA ---
def calcular_features_frequencia(sinal, taxa_amostral):
    """Calcula FF2, FF3 e FF5 de uma só vez."""
    N = len(sinal)
    if N == 0: return None, None, None

    # Aplica a FFT e obtém o espectro de amplitude
    espectro = np.abs(np.fft.fft(sinal)[0:N//2])
    freqs = np.fft.fftfreq(N, 1 / taxa_amostral)[:N//2]
    
    soma_espectro = np.sum(espectro)
    if soma_espectro == 0: return 0, 0, 0

    # FF2: Frequência Central
    ff2_freq_central = np.sum(freqs * espectro) / soma_espectro
    
    # FF3: RMS da Frequência
    ff3_rms_freq = np.sqrt(np.sum((freqs**2) * espectro) / soma_espectro)
    
    # FF4: Desvio Padrão da Frequência (necessário para FF5)
    ff4_std_freq = np.sqrt(np.sum(((freqs - ff2_freq_central)**2) * espectro) / soma_espectro)
    
    # FF5: Assimetria Espectral
    numerador_ff5 = np.sum(((freqs - ff2_freq_central)**3) * espectro) / soma_espectro
    ff5_assimetria = numerador_ff5 / (ff4_std_freq**3) if ff4_std_freq != 0 else 0
        
    return ff2_freq_central, ff3_rms_freq, ff5_assimetria


# --- 2. PROCESSAMENTO E EXTRAÇÃO DE FEATURES ---

print("--- Iniciando extração de atributos de cada SINAL SINTÉTICO (Drive End) ---")
lista_de_features = []

# Itera sobre o DataFrame com os sinais sintéticos do Drive End
# O método .itertuples() é mais rápido que iterrows()
for linha in df_final_de.itertuples():
    sinal_sintetico = linha.sinal_final
    
    # Calcula os atributos
    tf2 = calcular_tf2_std(sinal_sintetico)
    tf3 = calcular_tf3_rms(sinal_sintetico)
    tf4 = calcular_tf4_fator_forma(sinal_sintetico)
    ff2, ff3, ff5 = calcular_features_frequencia(sinal_sintetico, TAXA_AMOSTRAL)
    
    # Armazena os resultados junto com os metadados
    lista_de_features.append({
        'tipo_falha_adicionada': linha.tipo_falha_adicionada,
        'rpm': linha.rpm,
        'multiplicador_amplitude': linha.multiplicador_amplitude,
        'fase_adicionada_rad': linha.fase_adicionada_rad,
        'base_normal': linha.base_normal,
        'TF2_std': tf2,
        'TF3_rms': tf3,
        'TF4_fator_forma': tf4,
        'FF2_freq_central': ff2,
        'FF3_rms_freq': ff3,
        'FF5_assimetria_espectral': ff5
    })

# --- 3. CRIAÇÃO DO DATAFRAME FINAL DE ATRIBUTOS ---
df_features_sinteticas = pd.DataFrame(lista_de_features)

print(f"\nExtração concluída! Atributos de {len(df_features_sinteticas)} sinais sintéticos foram calculados.")
print("\n--- Exemplo do DataFrame final com os atributos extraídos ---")
print(df_features_sinteticas.head())

# Mostra informações sobre o DataFrame final
print("\n--- Informações do DataFrame de Atributos Sintéticos ---")
df_features_sinteticas.info()

--- Iniciando extração de atributos de cada SINAL SINTÉTICO (Drive End) ---

Extração concluída! Atributos de 169344 sinais sintéticos foram calculados.

--- Exemplo do DataFrame final com os atributos extraídos ---
  tipo_falha_adicionada   rpm  multiplicador_amplitude  fase_adicionada_rad  \
0         Pista Externa  1730                      0.1             0.000000   
1         Pista Externa  1730                      0.1             0.785398   
2         Pista Externa  1730                      0.1             1.570796   
3         Pista Externa  1730                      0.1             2.356194   
4         Pista Externa  1730                      0.1             3.141593   

            base_normal   TF2_std   TF3_rms  TF4_fator_forma  \
0  1730_Normal_FE_seg_0  0.073620  0.080450         1.256159   
1  1730_Normal_FE_seg_0  0.073620  0.080450         1.256160   
2  1730_Normal_FE_seg_0  0.073619  0.080450         1.256159   
3  1730_Normal_FE_seg_0  0.073619  0.080449         1